In [ ]:
# Imports
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
from PIL import Image as img
import math
from sklearn.cluster import KMeans
from scipy import stats


In [ ]:

BRYGGE_SEKVENS = "./bilder/brygge_sekvens"
BRO_SEKVENS = "./bilder/bro_sekvens"
LAGRA_BILDER = ".bilder/lagra_bilete"
BLANDA_SEKVENS = "./bilder/blanda_sekvens"
RESULT_FOLDER = BRYGGE_SEKVENS

K = np.loadtxt(f"{RESULT_FOLDER}/left/K_matrix.txt")
R = np.loadtxt(f"{RESULT_FOLDER}/left/R_matrix.txt")
T = np.loadtxt(f"{RESULT_FOLDER}/left/T_matrix.txt")

plt.ion()

# Under åpner vi ti = n'te bildet
left_images_filenames = list(filter(lambda fn: fn.split(".")[-1]=="png", os.listdir(f"{RESULT_FOLDER}/left")))
timestamps = list(map(lambda fn: fn.split(".")[0], left_images_filenames))
ti = 0
timestamp = timestamps[ti]
left = cv2.imread(f"{RESULT_FOLDER}/left/{timestamp}.png")
right = cv2.imread(f"{RESULT_FOLDER}/right/{timestamp}.png")
disp = np.array(cv2.imread(f"{RESULT_FOLDER}/disp_zed/{timestamp}.png", cv2.IMREAD_ANYDEPTH) / 256.0, dtype=np.float32)

print("Første:" ,timestamps[0], " Nærme: ", timestamps[len(timestamps)-1])

def fetch_image(main_as_well=False, RES_FOLDER=RESULT_FOLDER, time=timestamp):

    plt.ion()
    new_main_image = cv2.imread(f"{RES_FOLDER}/left/{time}.png")
    new_disparity_image = np.array(cv2.imread(f"{RES_FOLDER}/disp_zed/{time}.png", cv2.IMREAD_ANYDEPTH) / 256.0, dtype=np.float32)
    if(main_as_well):
        return new_disparity_image, new_main_image
    return new_disparity_image

def display_disp_image(to_be_displayed, title="Image", d_type="turbo"):
    plt.imshow(to_be_displayed, cmap=d_type)
    plt.title(title)
    plt.show()

# Timestamps
# Første: 1689072599961543541  Nærme:  1689072633388130541 Bru:  1689068851836122093 Båt: 1689068854381001093
Første= 1689072599961543541  
Nærme=  1689072633388130541
Bru =  1689068851836122093
Båt =  1689068854381001093

# Fra blanda
Kajakk1 = 1689068879500907093
KajakkOgBåt = 1689068879500907093
KajakkBåtTett = 1689068866505573093
BåtBak = 1689068856993399093
BåtFram = 1689068871596675093
BaderingMann = 1689072623607882541
Badering = 1689072630042776541
BaderingMannMidten = 1689072625349756541

 

# Fetch a new current working image and respective disparity
# Dette er disp_første og venstre_første
#current_working_disparity, current_working_image = fetch_image(True)

# Dette er disp_nærme_brygga og nærme_brygga
current_working_disparity, current_working_image = fetch_image(True, RES_FOLDER=BRYGGE_SEKVENS ,time=Første)
første_disparity, første_image = fetch_image(True, RES_FOLDER=BRYGGE_SEKVENS ,time=Første)
nærme_disparity, nærme_image = fetch_image(True, RES_FOLDER=BRYGGE_SEKVENS, time=Nærme)
bru_disparity, bru_image = fetch_image(True, RES_FOLDER=BRO_SEKVENS, time=Bru)
båt_disparity, båt_image = fetch_image(True, RES_FOLDER=BRO_SEKVENS, time=Båt)

kajakk_første_disparity, kajakk_første_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=Kajakk1)
kajakk_båt_disparity, kajakk_båt_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=KajakkOgBåt)
kajakk_tett_disparity, kajakk_tett_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=KajakkBåtTett)
båt_bak_disparity, båt_bak_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=BåtBak)
båt_fram_disparity, båt_fram_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=BåtFram)
badering_første_disparity, badering_første_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=Badering)
badering_mann_disparity, badering_mann_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=BaderingMann)
badering_midten_disparity, badering_midten_image = fetch_image(True, RES_FOLDER=BLANDA_SEKVENS ,time=BaderingMannMidten)


In [ ]:

def interpolate_column(column):
    # Indices of valid and invalid elements
    # Adds the indices of values that are valid meaning real numbers to valid_indices and invalid indices meaning inf, -inf and NaN to 
    valid_mask = np.isfinite(column)
    valid_indices = np.where(valid_mask)[0]
    invalid_indices = np.where(~valid_mask)[0]

    # Check if we have enough data for interpolation
    if len(valid_indices) == 0:
        # No valid data in this column
        return column
    elif len(invalid_indices) == 0:
        # No need for interpolation
        return column

    # Interpolate invalid data points
    valid_data = column[valid_mask]
    column[~valid_mask] = np.interp(invalid_indices, valid_indices, valid_data)
    return column

def interpolate_each_column(dI):
    # Applying the interpolation to each column
    height, width = dI.shape
    for x in range(width):
        dI[:, x] = interpolate_column(dI[:, x])

In [ ]:


n = 15  # Size of the kernel, as Vipul used in his thesis 5
kernel_k = np.ones(n) / n  # Kernel for averaging

# Gaussian kernel:
n_gaussian = 15  # Size of the kernel
sigma_kernel = 2.0  # Standard deviation which controls the spread

def gaussian_kernel(n_gaussian=n_gaussian, sigma=sigma_kernel): 
    # Create a coordinate vector from -n//2 to n//2
    x = np.arange(-n_gaussian//2 + 1, n_gaussian//2 + 1)

    # Calculate the Gaussian function
    kernel_g = np.exp(-0.5 * (x**2) / sigma**2)

    # Normalize the kernel to ensure the sum of kernel elements equals 1
    kernel_g /= np.sum(kernel_g)

    return kernel_g

kernel_gaussian = gaussian_kernel()

# Old binomial kernel
""" n_binomial = 15

def binomial_kernel(size = n_binomial):
    kernel_b = np.array([1, 1])
    for i in range(size - 2):
        kernel_b = np.convolve(kernel_b, np.array([1, 1]))
    kernel_b = kernel_b.astype(float)
    kernel_b /= np.sum(kernel_b)  # Normalize the kernel
    return kernel_b

kernel_binomial = binomial_kernel() """
from scipy.special import binom

n = 30  # Length of the kernel
kernel_binomial = np.array([binom(n-1, k) for k in range(n)])
kernel_binomial = kernel_binomial / np.sum(kernel_binomial) 

# Velg input Kernel!
# Endre/change/velg
#-------------------------------------------
#input_kernel_lowpass = kernel_k
#kern_bool = True
input_kernel_lowpass = kernel_binomial
kern_bool = False
#-------------------------------------------


# Function to apply low-pass filter to a column
def low_pass_filter(column, K=kernel_k):
    # Apply convolution
    filtered_column = np.convolve(column, K, mode='same')
    return filtered_column

#Endring av input, TELEPORTER hit
#-------------------------------------------
# Endre/change/velg
# Velg disparity bilde her!
#-------------------------------------------
#current_working_disparity, current_working_image 
#første_disparity, første_image 
#nærme_disparity, nærme_image 
#bru_disparity, bru_image 
#båt_disparity, båt_image
#
#dI = bru_disparity
#original_image = bru_image

#dI = båt_disparity
#original_image = båt_image

#dI = første_disparity
#original_image = første_image

#dI = nærme_disparity
#original_image = nærme_image

#dI = kajakk_første_disparity
#original_image = kajakk_første_image

#dI = kajakk_båt_disparity
#original_image = kajakk_båt_image

#dI = kajakk_tett_disparity
#original_image = kajakk_tett_image

dI = båt_bak_disparity
original_image = båt_bak_image

#dI = båt_fram_disparity
#original_image = båt_fram_image

#dI = badering_første_disparity
#original_image = badering_første_image

#dI = badering_mann_disparity
#original_image = badering_mann_image

#dI = badering_midten_disparity
#original_image = badering_midten_image


#-------------------------------------------



interpolate_each_column(dI)

# This is manually doing the above cell
# Apply the low-pass filter to each column
height, width = dI.shape
df = np.zeros_like(dI)  # Low-pass filtered disparity image
for x in range(width):
    df[:, x] = low_pass_filter(dI[:, x], K=input_kernel_lowpass)

#df = cv2.filter2D(dI, -1, input_kernel_lowpass.reshape(-1, 1))  # Reshape for vertical application

# Versjon 1 bruker sobel Kernel til å derivere
# Define the 1D Sobel kernelS
kernel_m = np.array([1, 0, -1])

# Initialize an array to store the vertical derivative image
vertical_derivative = np.zeros_like(df)

# Apply the Sobel kernel to each column
""" for i in range(df.shape[1]):  # Iterate over columns
    vertical_derivative[:, i] = np.convolve(df[:, i], kernel_m, mode='same') """

for i in range(df.shape[1]):  # Iterate over columns
    vertical_derivative[:, i] = np.convolve(df[:, i], kernel_m, mode='same') / 2



# Versjon 2 bruker np.diff 
# Computing the derivative of each column
df_derivative = np.diff(df, axis=0)

# Padding to handle size difference after np.diff
df_derivative = np.pad(df_derivative, ((0, 1), (0, 0)), mode='edge')
#første_df_derivative = np.pad(df_derivative, ((0, 1), (0, 0)), mode='edge') 



# Input should be a disparity image where each column is differentiated
def flatten_and_plot_histogram(df_derivative, plot=True,title="Histogram of Derivative Values", input_bins=2000):
    # Flatten the array of derivatives to a 1D array
    flattened_derivatives = df_derivative.flatten()

    if(plot):
        # Plot the histogram
        plt.figure()
        plt.hist(flattened_derivatives, bins=input_bins, range=(-0.3,0.5), color='blue', edgecolor='blue')
        plt.title(title)
        plt.xlabel('Derivative Value')
        plt.ylabel('Frequency')
        # Show the plot
        plt.show()
    
    return flattened_derivatives
    

# Versjon 1 Sobel
flattened_derivatives_sobel = flatten_and_plot_histogram(vertical_derivative, title="Sobel Method histogram")

# Versjon 2 np.diff
flattened_derivatives_diff = flatten_and_plot_histogram(df_derivative, title="Diff method histogram")

#flattened_derivatives = flattened_derivatives_sobel

flattened_derivatives = flattened_derivatives_diff

In [ ]:

def gaussian(x, mean, stddev):
    return (1 / (stddev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / stddev) ** 2)

# Gaussian fit function
def fit_gaussian(data):
    mean = np.mean(data)
    stddev = np.std(data)
    return mean, stddev

def clustering(flattened_derivatives, guess1=[0.0], guess2=[0.06], a_mini=-0.3, a_maxi=0.5, clusters=2):
    # Clipping the data to be between -0.3 and 0.5
    clipped_derivatives = np.clip(flattened_derivatives, a_min=a_mini, a_max=a_maxi)

    # Reshape data for KMeans
    clustering_model = clipped_derivatives.reshape(-1, 1)
    
    # Initial guesses for cluster centers (approximated from histogram peaks)
    initial_centers = np.array([guess1, guess2])  # replace peak1, peak2 with your estimates

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=clusters, init=initial_centers, n_init=1)
    kmeans.fit(clustering_model)

    # Get the labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    
    return clustering_model, clipped_derivatives, labels, centers

def clusteringV2(flattened_derivatives, a_mini=-0.3, a_maxi=0.5, clusters=2):
    # Clipping the data to be between -0.3 and 0.5
    clipped_derivatives = np.clip(flattened_derivatives, a_min=a_mini, a_max=a_maxi)

    # Reshape data for KMeans
    clustering_model = clipped_derivatives.reshape(-1, 1)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=clusters, init='k-means++', n_init=10)
    kmeans.fit(clustering_model)

    # Get the labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    
    return clustering_model, clipped_derivatives, labels, centers

# Sobel versjon
#clustering_model, labels, centers = clustering(flattened_derivatives_sobel, guess1=[0.0], guess2=[0.15])
# Diff
clustering_model, clipped_derivatives, labels, centers = clusteringV2(flattened_derivatives_diff)

# Relocation the mean to match the mode for a more accurate distribution

cluster1 = clustering_model[labels == 0]
cluster2 = clustering_model[labels == 1]

plt.figure()
plt.hist(cluster1, bins=200, range=(-0.2, 0.4), density=True, alpha=0.6)
plt.xlabel('Derivative Values')
plt.ylabel('Frequency')
plt.title('Cluster 1')
plt.legend()
plt.show()

# Plot histogram 
plt.figure()
plt.hist(cluster2, bins=200, range=(-0.2, 0.4), density=True, alpha=0.6)
plt.xlabel('Derivative Values')
plt.ylabel('Frequency')
plt.title('Cluster 2')
plt.legend()
plt.show()

plt.figure()
plt.hist(clustering_model, bins=200, range=(-0.2, 0.4), density=True, alpha=0.6)
plt.xlabel('Derivative Values')
plt.ylabel('Frequency')
plt.title('Histogram with Fitted Gaussian Curves')
plt.legend()
plt.show()

mode_result = stats.mode(cluster2)
print(type(mode_result.mode))

print("mode: ", stats.mode(cluster2).mode[0])



In [ ]:

# Fit Gaussian to each cluster
mean1, stddev1 = fit_gaussian(cluster1)
mean2, stddev2 = fit_gaussian(cluster2)
#stddev1, stddev2 = 0.02, 0.02
print("Mean 1: ",mean1)
print("Stddev 1: ",stddev1)
print("Mean 2: ",mean2)
print("Stddev 2: ",stddev2)
print("Mode 1: ", stats.mode(cluster1).mode[0])
print("Mode 2: ", stats.mode(cluster2).mode[0])

if(kern_bool):
    mean1 = stats.mode(cluster1).mode[0]
    mean2 = stats.mode(cluster2).mode[0]


# Determine which mean is closer to 0 and assign labels
if abs(mean1) < abs(mean2):
    red_mean, green_mean = mean1, mean2
    red_stddev, green_stddev = stddev1, stddev2
    red_label, green_label = 'Cluster 1 (Red)', 'Cluster 2 (Green)'
else:
    red_mean, green_mean = mean2, mean1
    red_stddev, green_stddev = stddev2, stddev1
    red_label, green_label = 'Cluster 2 (Red)', 'Cluster 1 (Green)'

# Below are the upper and lower bounds
# Assuming you have mean1, stddev1, mean2, stddev2 from the Gaussian fits
k = 1 # Adjust this constant as needed

""" shift_amount = 0.02  # Adjust this value to shift the curve to the left
# Adjust mean2 for Gaussian Curve 2
adjusted_mean2 = mean2 - shift_amount
mean2 = mean2 - shift_amount 
#gaussian_curve2Adjust = gaussian(x_values, adjusted_mean2, stddev2)
"""

# Calculate range limits for each Gaussian curve
dL1 = mean1 - k * stddev1
dU1 = mean1 + k * stddev1
dL2 = mean2 - k * stddev2
dU2 = mean2 + k * stddev2

red_dL = red_mean - k * red_stddev
red_dU = red_mean + k * red_stddev
green_dL = green_mean - k * green_stddev
green_dU = green_mean + k * green_stddev


# Create a range of x values
x_values = np.linspace(-0.3, 0.5, 400)



# Gaussian curves
gaussian_curve1 = gaussian(x_values, mean1, stddev1)
gaussian_curve2 = gaussian(x_values, mean2, stddev2)


# Plot histogram and Gaussian curves
plt.figure()
plt.hist(clipped_derivatives, bins=200, range=(-0.2, 0.4), density=True, alpha=0.6)
plt.plot(x_values, gaussian_curve1, label='Gaussian 1')
plt.plot(x_values, gaussian_curve2, label='Gaussian 2')
plt.xlabel('Derivative Values')
plt.ylabel('Frequency')
plt.title('Histogram with Fitted Gaussian Curves')
plt.legend()
plt.show()

In [ ]:
# And dL1, dU1, dL2, dU2 are the range limits for the two Gaussians
classification_map = np.empty(df_derivative.shape, dtype=object)

include_undecided_label = True

if include_undecided_label:
    print("includes undecided")
    for i in range(df_derivative.shape[0]):  # Rows
        for j in range(df_derivative.shape[1]):  # Columns
            derivative_value = df_derivative[i, j]

            # Check if the derivative value falls into the range of either Gaussian
            if (red_dL <= derivative_value <= red_dU) and (green_dL <= derivative_value <= green_dU):
                classification = 'undecided'
            elif red_dL <= derivative_value <= red_dU:
                classification = 'upright'  # Red cluster
            elif green_dL <= derivative_value <= green_dU:
                classification = 'horizontal'  # Green cluster
            else:
                classification = 'unknown'

            classification_map[i, j] = classification
else:
    print("Undecided and unknown is the same")
    for i in range(df_derivative.shape[0]):  # Rows
        for j in range(df_derivative.shape[1]):  # Columns
            derivative_value = df_derivative[i, j]

            # Check if the derivative value falls into the range of either Gaussian
            if (red_dL <= derivative_value <= red_dU) and (green_dL <= derivative_value <= green_dU):
                classification = 'unknown'
            elif red_dL <= derivative_value <= red_dU:
                classification = 'upright'  # Red cluster
            elif green_dL <= derivative_value <= green_dU:
                classification = 'horizontal'  # Green cluster
            else:
                classification = 'unknown'

            classification_map[i, j] = classification

print(df_derivative)
print(classification_map)

# Dark color definitions with brightness < 128/256
# Brightness = 0.18
Navy_Blue = [0, 0, 128/256]  # Brightness = 0.17
Dark_Slate_Gray = [47/256, 79/256, 79/256]  # Brightness = 0.32
Dark_Cyan = [0, 139/256, 139/256]  # Brightness = 0.18
Dark_Goldenrod = [184/256, 134/256, 11/256]  # Adjusted to be darker
White= [1,1,1]

# Revised dark, distinct color definitions
Dark_Red = [139/256, 0, 0]  # Rich, dark red
Dark_Slate_Gray = [47/256, 79/256, 79/256]
Sienna = [160/256, 82/256, 45/256]  # Earthy, medium dark
Dark_Orange = [255/256, 140/256, 0]  # Vibrant, but darkened
Saddle_Brown = [139/256, 69/256, 19/256] 
Pure_Red =[200/256,0,0]

# Horizontal color definitions
Indigo = [75/256, 0, 130/256]  # Deep indigo blue
Teal = [0, 128/256, 128/256]  # Rich teal blue
Forest_Green = [34/256, 139/256, 34/256]  # Deep forest green
Dark_Olive_Green = [85/256, 107/256, 47/256]  # Muted, darker green
Midnight_Blue = [25/256, 25/256, 112/256] 
Dark_Green = [0, 100/256, 0]  
Pure_Green = [0,200/256,0]
Pure_Blue = [0,0,200/256]


# Updated color scheme for classification ensuring maximum distinction
colorScheme = {
    'horizontal1': Navy_Blue,
    'horizontal2': Teal,
    'horizontal3': Pure_Green,
    'horizontal4': Indigo,
    'horizontal5': Dark_Olive_Green,
    'horizontal6': Pure_Green,  # Custom darker teal
    'horizontal7': Dark_Green,  # Custom dark leaf green
    'vertical1': Dark_Red,
    'vertical2': Pure_Red,
    'vertical3': Sienna,
    'vertical4': Dark_Orange,
    'vertical5': Dark_Slate_Gray,  # Custom burnt orange
    'unknown': White,  # White
    'undecided': White,  # White
    'upright': Pure_Red,
    'horizontal': Pure_Green
}


def assign_colours_and_plot(classification_map, colors=colorScheme, plot=True, title="Pixel-wise Disparity Map Classification"):
    # Create an empty array for the color-coded image
    color_coded_image = np.zeros((*classification_map.shape, 3))

    # Assign colors
    for classification, color in colors.items():
        mask = classification_map == classification
        color_coded_image[mask] = color

    if(plot):
        plt.figure(figsize=(10, 6))
        plt.imshow(color_coded_image)
        plt.title(title)
        plt.axis('off')  # Hide the axes
        plt.show()



# Nærme
#display_disp_image(nærme_image)
# Båt
#display_disp_image(båt_test_bilde)
assign_colours_and_plot(classification_map, title="Horizontal and vertical pixel segmentation")
display_disp_image(original_image)


In [ ]:
surface_splitting_disparity_image = dI

vertCount = 0
horCount = 0
noneCount = 0

# Verticals
vertical_Derivative_List = []
vertical_Disp_List = []
vertical_Obj_List = []


#Horizonals
horizontal_Derivative_List = []
horizontal_Disp_List = []
horizontal_Obj_List = []

divided_horizontal_obj_list = []


for i in range(classification_map.shape[0]):  # Rows

    single_col_obj_list = []
    for j in range(classification_map.shape[1]):  # Columns

        current_local_value = classification_map[i, j]

        if(current_local_value == 'upright'):
            vertCount += 1
            vertical_Derivative_List.append(df_derivative[i,j])
            appendObjectVert = [surface_splitting_disparity_image[i, j], i, j, 'vertical']
            vertical_Disp_List.append(surface_splitting_disparity_image[i,j])
            vertical_Obj_List.append(appendObjectVert)
        
        
        
        elif(current_local_value == 'horizontal'):
            horCount += 1
            der_val = df_derivative[i, j]
            horizontal_Derivative_List.append(der_val)
            # Object = [Disparity value, vertical derivative value, i coordinate, j coordinate, label]
            appendObjectHor = [surface_splitting_disparity_image[i, j], der_val, i, j, 'horizontal']
            horizontal_Disp_List.append(surface_splitting_disparity_image[i,j])
            horizontal_Obj_List.append(appendObjectHor)
            single_col_obj_list.append(appendObjectHor)

        else:
            noneCount+=1
    divided_horizontal_obj_list.append(single_col_obj_list)

print("verticals: ", vertCount)
print("Horizontals: ", horCount)
print("Unknown: ", noneCount)
print("[Disparity Value, Derivative Value, V index, U index, Label]")
print(horizontal_Obj_List[202])

In [ ]:
# Assuming vertical_Disp_List is a list of disparity values
vertical_Disp_List = np.array(vertical_Disp_List)  # Convert list to NumPy array for efficient operations

# Adding higher percentiles (97%, 98%, 99%)
p97 = np.percentile(vertical_Disp_List, 97)
p3 = np.percentile(vertical_Disp_List, 3)
filtered_data_97 = vertical_Disp_List[(vertical_Disp_List >= p3) & (vertical_Disp_List <= p97)]

p98 = np.percentile(vertical_Disp_List, 98)
p2 = np.percentile(vertical_Disp_List, 2)
filtered_data_98 = vertical_Disp_List[(vertical_Disp_List >= p2) & (vertical_Disp_List <= p98)]
filtered_data_2 = vertical_Disp_List[(vertical_Disp_List < p2) | (vertical_Disp_List > p98)]

p99 = np.percentile(vertical_Disp_List, 99)
p1 = np.percentile(vertical_Disp_List, 1)
filtered_data_99 = vertical_Disp_List[(vertical_Disp_List >= p1) & (vertical_Disp_List <= p99)]

len_97 = len(filtered_data_97)
len_98 = len(filtered_data_98)
len_99 = len(filtered_data_99)


print("Length of data capturing 97%: ", len_97)
print("Length of data capturing 98%: ", len_98)
print("Length of data capturing 99%: ", len_99)

min_97 = min(filtered_data_97)
max_97 = max(filtered_data_97)

min_98 = min(filtered_data_98)
max_98 = max(filtered_data_98)

min_99 = min(filtered_data_99)
max_99 = max(filtered_data_99)

min_2 = min(filtered_data_2)
max_2 = max(filtered_data_2)

print("Min: ", min_97, " and max: ", max_97, " for values within 97%")
print("Min: ", min_98, " and max: ", max_98, " for values within 98%")
print("Min ", min_99, " and max: ", max_99, " for values within 99%")

print("")
print("Min: ", min_2, " and max: ", max_2, " for values outside 98%")

In [ ]:
from scipy.signal import find_peaks
auto_range=(-1,60)
def plot_histogram_with_peaks(data, bins, min_prominence=0.01, min_distance=1, auto_range=auto_range):

    # Ensure data is cleaned
    data = np.array(data)
    data = data[np.isfinite(data)]


    hist, bin_edges = np.histogram(data,range=auto_range, bins=bins, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Get bin centers for plotting

    # Find peaks with adjusted parameters
    peaks, properties = find_peaks(hist, prominence=min_prominence, distance=min_distance)

    # Plotting the histogram and peaks
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=bins,  range=auto_range, alpha=0.6, label='Histogram', density=True)
    plt.plot(bin_centers, hist, label='Histogram Line')
    plt.scatter(bin_centers[peaks], hist[peaks],color='red', s=100, zorder=5, label='Peaks/Modes')
    plt.title('Cluster Data and Detected Modes')
    plt.xlabel('Data Value')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

    # Print peak bin centers
    peak_values = bin_centers[peaks]
    print("Detected modes at:", peak_values)
    return peak_values

# Call the function with parameters adjusted based on your description
detected_modes = plot_histogram_with_peaks(filtered_data_98, bins=100, min_prominence=0.01, min_distance=1)
detected_modes = plot_histogram_with_peaks(vertical_Disp_List, bins=100, min_prominence=0.01, min_distance=1)




In [ ]:
def clusteringVerticalSurfaces(flattened_derivatives, a_mini=-0.3, a_maxi=50, clusters=4):
    # Clipping the data to be between -0.3 and 0.5
    clipped_derivatives = np.clip(flattened_derivatives, a_min=a_mini, a_max=a_maxi)

    # Reshape data for KMeans
    clustering_model = clipped_derivatives.reshape(-1, 1)    

    #kmeans = KMeans(n_clusters=clusters, init=initial_centers, n_init=1)
    kmeans = KMeans(n_clusters=clusters, init='k-means++', n_init=10)
    kmeans.fit(clustering_model)

    # Get the labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    
    return clustering_model, clipped_derivatives, labels, centers

num_clusters = len(detected_modes)

#model_choice = "Standard"
#model_choice= "Simple_98"
model_choice = "Advanced_98"

# Standard model
if(model_choice == "Standard"):
    clustering_model, clipped_derivatives, labels, centers = clusteringVerticalSurfaces(vertical_Disp_List, clusters=num_clusters)

# 98th percentile model
if(model_choice = "Simple_98"):
    clustering_model, clipped_derivatives, labels, centers = clusteringVerticalSurfaces(vertical_Disp_List, clusters=num_clusters, a_mini=min_98, a_maxi=max_98)

# 98th percentile + Unknown labelings
if(model_choice =="Advanced_98"):
    clustering_model, clipped_derivatives, labels, centers = clusteringVerticalSurfaces(filtered_data_98, clusters=num_clusters, a_mini=min_98, a_maxi=max_98)




# Separate the data into two clusters
print("Number og clusters: ", num_clusters)
if(num_clusters>= 1):
    cluster1 = clustering_model[labels == 0]
if(num_clusters>= 2):
    cluster2 = clustering_model[labels == 1]
if(num_clusters >= 3):
    cluster3 = clustering_model[labels == 2]
if(num_clusters >= 4):
    cluster4 = clustering_model[labels == 3]
if(num_clusters >= 5):
    cluster5 = clustering_model[labels == 4]


# Plot histogram and Gaussian curves
plt.figure()
if(num_clusters>= 1):
    plt.hist(cluster1, bins=200, range=(0, 50), density=True, alpha=0.6, color='blue', edgecolor='blue')
if(num_clusters>= 2):
    plt.hist(cluster2, bins=200, range=(0, 50), density=True, alpha=0.6, color='red', edgecolor='red')
if(num_clusters>= 3):
    plt.hist(cluster3, bins=200, range=(0, 50), density=True, alpha=0.6, color='green', edgecolor='green')
if(num_clusters>= 4):
    plt.hist(cluster4, bins=200, range=(0, 50), density=True, alpha=0.6, color='yellow', edgecolor='yellow')
if(num_clusters>= 5):
    plt.hist(cluster5, bins=200, range=(0, 50), density=True, alpha=0.6, color='purple', edgecolor='purple')
#plt.plot(x_values, gaussian_curve1, label='Gaussian 1')
#plt.plot(x_values, gaussian_curve2, label='Gaussian 2')
plt.xlabel('Derivative Values')
plt.ylabel('Frequency')
plt.title('Histogram with Fitted Gaussian Curves')
plt.legend()
plt.show()

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Create classmap
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------

for i, obj in enumerate(vertical_Obj_List):
    if(labels[i] == 0):
        obj[3] = "vertical1"
    elif(labels[i] == 1):
        obj[3] = "vertical2"
    elif(labels[i] == 2):
        obj[3] = "vertical3"
    elif(labels[i] == 3):
        obj[3] = "vertical4"
    elif(labels[i]==4):
        obj[3] = "vertical5"
    else:
        obj[3] = "unknown"


new_class_map = np.copy(classification_map)

for i, obj in enumerate(vertical_Obj_List):
    i = obj[1]
    j= obj[2]
    new_class_map[i,j] = obj[3]


""" # Dark color definitions with brightness < 128/256
# Brightness = 0.18
Navy_Blue = [0, 0, 128/256]  # Brightness = 0.17
Dark_Slate_Gray = [47/256, 79/256, 79/256]  # Brightness = 0.32
Dark_Cyan = [0, 139/256, 139/256]  # Brightness = 0.18
Dark_Goldenrod = [184/256, 134/256, 11/256]  # Adjusted to be darker
White= [1,1,1]

# Revised dark, distinct color definitions
Dark_Red = [139/256, 0, 0]  # Rich, dark red
Dark_Slate_Gray = [47/256, 79/256, 79/256]
Sienna = [160/256, 82/256, 45/256]  # Earthy, medium dark
Dark_Orange = [255/256, 140/256, 0]  # Vibrant, but darkened
Saddle_Brown = [139/256, 69/256, 19/256] 
Pure_Red =[200/256,0,0]

# Horizontal color definitions
Indigo = [75/256, 0, 130/256]  # Deep indigo blue
Teal = [0, 128/256, 128/256]  # Rich teal blue
Forest_Green = [34/256, 139/256, 34/256]  # Deep forest green
Dark_Olive_Green = [85/256, 107/256, 47/256]  # Muted, darker green
Purple = [128/256, 0, 128/256]  # Rich deep purple
Midnight_Blue = [25/256, 25/256, 112/256] 
Dark_Green = [0, 100/256, 0]  
Pure_Green = [0,200/256,0]
Pure_Blue = [0,0,200/256]


# Updated color scheme for classification ensuring maximum distinction
colorScheme = {
    'horizontal1': Navy_Blue,
    'horizontal2': Teal,
    'horizontal3': Pure_Green,
    'horizontal4': Indigo,
    'horizontal5': Dark_Olive_Green,
    'horizontal6': Pure_Green,  # Custom darker teal
    'horizontal7': Dark_Green,  # Custom dark leaf green
    'vertical1': Dark_Red,
    'vertical2': Pure_Red,
    'vertical3': Sienna,
    'vertical4': Dark_Orange,
    'vertical5': Dark_Slate_Gray,  # Custom burnt orange
    'unknown': White,  # White
    'undecided': White,  # White
    'upright': Pure_Red,
    'horizontal': Pure_Green
}"""


# Create an empty array for the color-coded image
color_coded_image = np.zeros((*new_class_map.shape, 3))
#print(color_coded_image)


color_coded_image = np.zeros((*new_class_map.shape, 3))

# Assign colors based on the classification in new_class_map
for label, color in colorScheme.items():
    # Find indices where the current label is present in new_class_map
    indices = np.where(new_class_map == label)
    # Assign the corresponding color to these indices in the color-coded image
    color_coded_image[indices] = color

#print(color_coded_image)
plt.figure(figsize=(10, 6))
plt.imshow(color_coded_image)
plt.title('Pixel-wise Disparity Map Classification')
plt.axis('off')  # Hide the axes
plt.show() 

In [ ]:
print("Cluster1: ", (cluster1))
print("2 percent list: ",type(filtered_data_2))
print("Labels: ", (labels[i]))

In [ ]:
# Endre/change/velg
# Velg column index here
column_index = 500

#Total rows (v) up = 1080
#Total columns (u) right = 1920

# Endre/change/velg
# Velg column data here
column_data = surface_splitting_disparity_image[:, column_index]
y_values = range(len(column_data))



print(surface_splitting_disparity_image[:, column_index])
print(df_derivative[:, column_index])
print(classification_map[:, column_index])

count = 0
#horizontal_column_disparity_list = []



def single_column_list_generator(column_index=column_index, only_return_hor_col=False):

    horizontal_column_derivative_list = []
    horizontal_column_object_list = []
    vertical_column_derivative_list = []
    vertical_column_object_list = []
    for i, val in enumerate(classification_map[:, column_index]):
        #print(val)
        
        if(val == 'horizontal'):
            der = df_derivative[i, column_index]
            disp = surface_splitting_disparity_image[i, column_index]

            appendObjColHor = [der, disp, i, column_index, val]
            horizontal_column_derivative_list.append(der)
            horizontal_column_object_list.append(appendObjColHor)

        #if(val == 'vertical1' or val == 'vertical2' or val == 'vertical3' or val == 'vertical4'):
        if(val == 'upright'):
            der = df_derivative[i, column_index]
            disp = surface_splitting_disparity_image[i, column_index]

            appendObjColVert = [der, disp, i, column_index, val]
            vertical_column_derivative_list.append(der)
            vertical_column_object_list.append(appendObjColVert)
    if(only_return_hor_col):
        return horizontal_column_object_list
    return horizontal_column_derivative_list, horizontal_column_object_list, vertical_column_derivative_list, vertical_column_object_list

horizontal_column_derivative_list, horizontal_column_object_list, vertical_column_derivative_list, vertical_column_object_list = single_column_list_generator()

def generate_index_list(column_object_list=horizontal_column_object_list):
    indexListColumn = []
    for i in range(len(column_object_list)):
        ind = column_object_list[i][2]
        indexListColumn.append(ind)
    return indexListColumn




# Index list horCOl
print(len(horizontal_column_object_list))
indexListHorCol = []
for i in range(len(horizontal_column_object_list)):
    ind = horizontal_column_object_list[i][2]
    indexListHorCol.append(ind)
    #print("Index ", ind)
print("IndexListHorCol: ",indexListHorCol)


# Index list vertCol
print(len(vertical_column_object_list))
indexListVertCol = []
for i in range(len(vertical_column_object_list)):
    ind = vertical_column_object_list[i][2]
    indexListVertCol.append(ind)
    #print("Index ", ind)
print("IndexListVertCol: ",indexListVertCol)




plt.figure()
#plt.plot(y_values, column_data)

highlighted_horizontal_y_values = [y_values[i] for i in indexListHorCol]
highlighted_horizontal_column_data = [column_data[i] for i in indexListHorCol]

highlighted_vertical_y_values = [y_values[i] for i in indexListVertCol]
highlighted_vertical_column_data = [column_data[i] for i in indexListVertCol]



# Plotting highlighted points on top with a different color
plt.scatter(highlighted_vertical_y_values, highlighted_vertical_column_data, color='red', s=2, zorder=1, label='Highlighted Points')
plt.scatter(highlighted_horizontal_y_values, highlighted_horizontal_column_data, color='green', s=2, zorder=1, label='Highlighted Points')

title_for_plot= 'Column index: ' + str(column_index)
plt.gca()
plt.ylabel('Disparity Value')
plt.xlabel('Image v-Axis')
plt.title(title_for_plot)
plt.show()

import matplotlib.colors as mcolors

def generate_dark_colors(n):
    # Define the saturation and value for dark colors
    saturation = 1.0  # full saturation for vivid colors
    value = 0.6  # dark enough to ensure good contrast on a white background
    
    # Generate colors
    colors = []
    for i in range(n):
        hue = i / n  # Evenly space hues around the color wheel
        rgb = mcolors.hsv_to_rgb((hue, saturation, value))
        colors.append(rgb)
    
    return colors

# Example usage: generate 10 dark, distinct colors
num_colors = 10
dark_colors = generate_dark_colors(num_colors)


In [ ]:
def group_indices(indices, limit=10):
    if not indices:
        return []
    sorted_indices = sorted(indices)
    groups = [[sorted_indices[0]]]
    for index in sorted_indices[1:]:
        if index - groups[-1][-1] <= limit:
            groups[-1].append(index)
        else:
            groups.append([index])
    return groups

def perform_linear_regression(groups, y_values, column_data):
    line_equations = []
    edge_boundaries = []  # List to store edge boundaries for each group
    min_size = 10
    for group in groups:
        if len(group) >= min_size:
            group_y_values = np.array([y_values[idx] for idx in group])
            group_column_data = np.array([column_data[idx] for idx in group])
            A = np.vstack([group_y_values, np.ones(len(group_y_values))]).T
            m, c = np.linalg.lstsq(A, group_column_data, rcond=None)[0]
            line_equations.append((m, c))
            # Store the min and max v-values for the group
            edge_boundaries.append((min(group_y_values), max(group_y_values)))
        else:
            line_equations.append(None)
            edge_boundaries.append(None)
    return line_equations, edge_boundaries

def reclassify_points_and_identify_edges(indexList, column_data, line_equations, edge_boundaries, column_index):
    reclassified_points = []
    edge_points = {}
    corrected_points = []  # New list to store corrected points
    unknown_points = []

    for idx, v_value in enumerate(indexList):
        u_value = column_index
        disparity = column_data[v_value]
        
        # Function to calculate distance to line
        def distance_to_line(line):
            m, c = line
            return abs(m * v_value - disparity + c) / np.sqrt(m**2 + 1)
        
        # Create a sorted list of lines based on their distance to the point
        valid_lines = [(line, boundaries) for line, boundaries in zip(line_equations, edge_boundaries) if line is not None and boundaries is not None]
        sorted_lines = sorted(valid_lines, key=lambda x: distance_to_line(x[0]))

        # Try to classify the point to the first valid line within boundaries
        classified = False
        for line, (min_v, max_v) in sorted_lines:
            if min_v <= v_value <= max_v:
                reclassified_points.append((u_value, v_value, disparity, line))
                m, c = line
                corrected_disparity = m * v_value + c  # y = mx + c
                corrected_points.append((u_value, v_value, corrected_disparity, line))
                classified = True
                break

        # Update edge points if the point was classified
        if classified:
            if line not in edge_points:
                edge_points[line] = {'min_v': (float('inf'), None), 'max_v': (float('-inf'), None)}
            if v_value < edge_points[line]['min_v'][0]:
                edge_points[line]['min_v'] = (v_value, (u_value, v_value, disparity))
            if v_value > edge_points[line]['max_v'][0]:
                edge_points[line]['max_v'] = (v_value, (u_value, v_value, disparity))
        else:
            # If no valid line was found to classify the point, add it to the unknown points list
            unknown_points.append((u_value, v_value, disparity, "unknown"))

    return reclassified_points, edge_points, corrected_points, unknown_points


size_of_fig=(10, 6)
# Main function to process column data
def process_column(column_index, plot=False, size_of_fig=size_of_fig, group_limit=10):
    column_data = surface_splitting_disparity_image[:, column_index]
    y_values = range(len(column_data))

    horizontal_column_object_list = single_column_list_generator(column_index, True)
    indexListHorCol = generate_index_list(horizontal_column_object_list)


    groups = group_indices(indexListHorCol, limit=group_limit)
    colors = generate_dark_colors(len(groups))

    
    line_equations, edge_boundaries = perform_linear_regression(groups, list(y_values), column_data)
    reclassified_points, edge_points, corrected_points, unknown_points = reclassify_points_and_identify_edges(indexListHorCol, column_data, line_equations, edge_boundaries, column_index)

    if plot:
        # Plot 1: Grouping and Regression
        plt.figure(figsize=size_of_fig)
        for i, group in enumerate(groups):
            if group:
                group_y_values = [y_values[idx] for idx in group]
                group_column_data = [column_data[idx] for idx in group]
                plt.scatter(group_y_values, group_column_data, color=colors[i], s=10, label=f'Group {i+1}')
                
                if line_equations[i]:
                    m, c = line_equations[i]
                    # Calculate extended x values but make them shorter than the full range
                    extended_start = min(group_y_values) - 100  # you can adjust these values to control the length
                    extended_end = max(group_y_values) + 100   # likewise, adjust here for the end
                    extended_x = np.linspace(extended_start, extended_end, 400)
                    plt.plot(extended_x, m * extended_x + c, color=colors[i], label=f'Line: y={m:.2f}x+{c:.2f}')

        plt.xlabel('Image v-Axis')
        plt.ylabel('Disparity Value')
        plt.title('Grouping and Regression for Column ' + str(column_index))
        plt.legend()
        plt.show()

        # Plot 2: Reclassified Points
        line_to_color = {line: colors[i] for i, line in enumerate(line_equations) if line is not None}

        plt.figure(figsize=(10, 6))
        for point in reclassified_points:
            u_value, v_value, disparity, line = point
            if line in line_to_color:
                color = line_to_color[line]
                plt.scatter(v_value, disparity, color=color, s=10)  # Use the color mapped to the line

        for point in unknown_points:
            u_value, v_value, disparity, line = point
            plt.scatter(v_value, disparity, color='black', s=10)

        plt.xlabel('Image v-Axis')
        plt.ylabel('Disparity Value')
        plt.title('Reclassified Points for Column ' + str(column_index))
        plt.show()

        # Plot 3: Edge Points
        plt.figure(figsize=size_of_fig)
        line_to_color = {line: colors[i] for i, line in enumerate(line_equations) if line is not None}
        for line, edge in edge_points.items():
            if line in line_to_color:
                color = line_to_color[line]  # Use the mapped color for the line
                for key, value in edge.items():
                    if value[1] is not None:
                        plt.scatter(value[1][1], value[1][2], color=color, s=10, label=f'{key} Edge Point')
        for point in unknown_points:
            u_value, v_value, disparity, line = point
            plt.scatter(v_value, disparity, color='black', s=10)
        plt.xlabel('Image v-Axis')
        plt.ylabel('Disparity Value')
        plt.title('Edge Points for Column ' + str(column_index))
        plt.legend()
        plt.show()

   
        # Plot corrected points as an additional plot or integrated into one of the existing plots
        plt.figure(figsize=size_of_fig)
        for point in corrected_points:
            u_value, v_value, disparity, line = point
            if line in line_to_color:
                color = line_to_color[line]
                plt.scatter(v_value, disparity, color=color, s=10) 
        for point in unknown_points:
            u_value, v_value, disparity, line = point
            plt.scatter(v_value, disparity, color='black', s=10)
        plt.xlabel('Image v-Axis')
        plt.ylabel('Corrected Disparity Value')
        plt.title('Corrected Points for Column ' + str(column_index))
        plt.legend()
        plt.show()

    return reclassified_points, edge_points, corrected_points, unknown_points


def group_corrected_points_by_line(corrected_points, unknown_points):
    line_groups = {}
    for point in corrected_points:
        u_value, v_value, corrected_disparity, line = point
        line_key = tuple(line)  # Use the line equation as the dictionary key
        if line_key not in line_groups:
            line_groups[line_key] = []
        line_groups[line_key].append([u_value, v_value, corrected_disparity])

    # Handle unknown points
    unknown_key = ('unknown',)
    line_groups[unknown_key] = []
    for point in unknown_points:
        u_value, v_value, disparity, _ = point
        line_groups[unknown_key].append([u_value, v_value, disparity])

    return list(line_groups.values())



horizontal_label_list = ['horizontal1', 'horizontal2', 'horizontal3', 'horizontal4', 'horizontal5', 'horizontal6', 'horizontal7']

def new_column_object_list_generator(grouped_corrected_points, new_full_object_list=[]):

    new_column_object_list = []

    last_group = len(grouped_corrected_points)-1
    #print(last_group)

    for i, group in enumerate(reversed(grouped_corrected_points)):
        for point in group:
            if(i>= len(horizontal_label_list) or (i ==0)):
                point.append('unknown')
            else:
                point.append(horizontal_label_list[i-1])
            #print("Point: ", point)
            new_column_object_list.append(point)
            new_full_object_list.append(point)

    return new_column_object_list, new_full_object_list

# Example of how to call this function
current_column_index = 1500  # Example column index
reclassified_points_initial, edge_points_initial, corrected_points, unknown_points =  process_column(current_column_index, plot=True, group_limit=20)

grouped_corrected_points = group_corrected_points_by_line(corrected_points, unknown_points)
newest_test_list, disregarded= new_column_object_list_generator(grouped_corrected_points)

#print("Length ",len(newest_test_list))


for i, group in enumerate(grouped_corrected_points):
    print("index: ", i)
    print("Group:", group) 


In [ ]:
# Takes 15 secs to run
def all_columns_full_horizontal_object_list_generator(surface_splitting_disparity_image, plot=False, group_limit=10):
    all_columns_full_horizontal_object_list = []
    
    # Assume there is a known number of columns
    total_columns = surface_splitting_disparity_image.shape[1]

    for current_column_index in range(total_columns):
        reclassified_points_initial, edge_points_initial, corrected_points, unknown_points =  process_column(current_column_index, plot=False,group_limit=group_limit)
        grouped_corrected_points = group_corrected_points_by_line(corrected_points, unknown_points)
        newest_test_list, all_columns_full_horizontal_object_list = new_column_object_list_generator(grouped_corrected_points, all_columns_full_horizontal_object_list)
        
    return all_columns_full_horizontal_object_list

all_columns_full_horizontal_object_list = all_columns_full_horizontal_object_list_generator(surface_splitting_disparity_image, group_limit=20)
print("Length all columns",len(all_columns_full_horizontal_object_list))
print("Length obj list", len(horizontal_Obj_List))

In [ ]:
horizontal_test_class_map = np.copy(classification_map)
FULL_TEST_CLASS_MAP = np.copy(new_class_map)


for index, obj in enumerate(all_columns_full_horizontal_object_list):
    i = obj[1]
    j= obj[0]
    lab = obj[3]   
    horizontal_test_class_map[i,j] = obj[3]
    FULL_TEST_CLASS_MAP[i, j] = obj[3]
""" 
# Dark color definitions with brightness < 128/256
# Brightness = 0.18
Navy_Blue = [0, 0, 128/256]  # Brightness = 0.17
Dark_Slate_Gray = [47/256, 79/256, 79/256]  # Brightness = 0.32
Dark_Cyan = [0, 139/256, 139/256]  # Brightness = 0.18
Dark_Goldenrod = [184/256, 134/256, 11/256]  # Adjusted to be darker
White= [1,1,1]

# Revised dark, distinct color definitions
Dark_Red = [139/256, 0, 0]  # Rich, dark red
Dark_Slate_Gray = [47/256, 79/256, 79/256]
Sienna = [160/256, 82/256, 45/256]  # Earthy, medium dark
Dark_Orange = [255/256, 140/256, 0]  # Vibrant, but darkened
Saddle_Brown = [139/256, 69/256, 19/256] 
Pure_Red =[200/256,0,0]

# Horizontal color definitions
Indigo = [75/256, 0, 130/256]  # Deep indigo blue
Teal = [0, 128/256, 128/256]  # Rich teal blue
Forest_Green = [34/256, 139/256, 34/256]  # Deep forest green
Dark_Olive_Green = [85/256, 107/256, 47/256]  # Muted, darker green
Midnight_Blue = [25/256, 25/256, 112/256] 
Dark_Green = [0, 100/256, 0]  
Pure_Green = [0,200/256,0]
Pure_Blue = [0,0,200/256]


# Updated color scheme for classification ensuring maximum distinction
colorScheme = {
    'horizontal1': Navy_Blue,
    'horizontal2': Teal,
    'horizontal3': Pure_Green,
    'horizontal4': Indigo,
    'horizontal5': Dark_Olive_Green,
    'horizontal6': Pure_Green,  # Custom darker teal
    'horizontal7': Dark_Green,  # Custom dark leaf green
    'vertical1': Dark_Red,
    'vertical2': Pure_Red,
    'vertical3': Sienna,
    'vertical4': Dark_Orange,
    'vertical5': Dark_Slate_Gray,  # Custom burnt orange
    'unknown': White,  # White
    'undecided': White,  # White
    'upright': Pure_Red,
    'horizontal': Pure_Blue
}
 """

# Create an empty array for the color-coded image
horizontal_color_coded_image = np.zeros((*horizontal_test_class_map.shape, 3))
#print(color_coded_image)
full_color_coded_image = np.zeros((*FULL_TEST_CLASS_MAP.shape, 3))


# Assign colors based on the classification in new_class_map
for label, color in colorScheme.items():
    # Find indices where the current label is present in new_class_map
    indices = np.where(horizontal_test_class_map == label)
    fulldices= np.where(FULL_TEST_CLASS_MAP==label)
    # Assign the corresponding color to these indices in the color-coded image
    horizontal_color_coded_image[indices] = color
    full_color_coded_image[fulldices] = color

#print(color_coded_image)
plt.figure(figsize=(10, 6))
plt.imshow(horizontal_color_coded_image)
plt.title('Pixel-wise Disparity Map Classification')
plt.axis('off')  # Hide the axes
plt.show() 
plt.figure(figsize=(10, 6))
plt.imshow(color_coded_image)
plt.title('Pixel-wise Disparity Map Classification')
plt.axis('off')  # Hide the axes
plt.show() 
plt.figure(figsize=(10, 6))
plt.imshow(full_color_coded_image)
plt.title('Pixel-wise Disparity Map Classification')
plt.axis('off')  # Hide the axes
plt.show() 

display_disp_image(original_image)

In [ ]:
# TELEPORTER